In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# Load CSV file
data = pd.read_csv('Roman-Urdu-Poetry.csv')

# Filter the first 800 rows for training
data = data.head(800)

# Extract the poetry column
poetry = data['Poetry'].values

# Tokenize the poetry text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(poetry)

# Convert texts to sequences of integers
sequences = tokenizer.texts_to_sequences(poetry)

# Get the maximum sequence length
max_len = max([len(seq) for seq in sequences])

# Pad sequences to have equal length
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

# Split the sequences into input (X) and output (y)
X = sequences[:, :-1]  # Input: all words except last
y = sequences[:, -1]   # Output: last word (target)

# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Ensure correct input data and vocab size
print(f"X.shape: {X.shape}")
print(f"vocab_size: {vocab_size}")

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=X.shape[1]))  # Correctly pass input_length
model.add(LSTM(128, return_sequences=False))
model.add(Dense(vocab_size, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()


X.shape: (800, 565)
vocab_size: 8386


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
model.fit(X, y, epochs=80, batch_size=32)


Epoch 1/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6382 - loss: 1.8512
Epoch 2/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6687 - loss: 1.7034
Epoch 3/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6719 - loss: 1.7650
Epoch 4/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6759 - loss: 1.6612
Epoch 5/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7061 - loss: 1.6247
Epoch 6/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7219 - loss: 1.5972
Epoch 7/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7335 - loss: 1.5616
Epoch 8/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7742 - loss: 1.4156
Epoch 9/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7772 - loss: 1.4364
Epoch 10/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7900 - loss: 1.3752
Epoch 11/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7949 - loss: 1.3341
Epoch 12/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy:

In [7]:
# Save the trained model
model.save('poetry_model.h5')


In [19]:
import numpy as np

def top_k_sampling(pred_probs, k=5):
    """Select the top-k most likely next words and sample from them."""
    top_k_indices = np.argsort(pred_probs)[-k:]  # Get the top k indices
    top_k_probs = pred_probs[top_k_indices]  # Get the corresponding probabilities
    top_k_probs = top_k_probs / np.sum(top_k_probs)  # Normalize to get a valid probability distribution
    return np.random.choice(top_k_indices, p=top_k_probs)  # Sample from top k words

def top_p_sampling(pred_probs, p=0.9):
    """Select the top-p most likely next words (nucleus sampling)."""
    sorted_indices = np.argsort(pred_probs)[::-1]  # Sort probabilities in descending order
    sorted_probs = pred_probs[sorted_indices]
    cumulative_probs = np.cumsum(sorted_probs)

    # Find the cutoff index where the cumulative probability exceeds p
    cutoff_index = np.where(cumulative_probs >= p)[0][0] + 1
    filtered_indices = sorted_indices[:cutoff_index]
    filtered_probs = sorted_probs[:cutoff_index]
    filtered_probs = filtered_probs / np.sum(filtered_probs)  # Normalize to get a valid probability distribution

    return np.random.choice(filtered_indices, p=filtered_probs)  # Sample from top p words

def generate_poetry(model, tokenizer, start_word, num_words_per_line=10, num_lines=3, temperature=1.0, sampling_method='top_k', k=5, p=0.9):
    # Initialize the poem with the starting word and the first line
    poem = start_word
    sequence = tokenizer.texts_to_sequences([start_word])
    sequence = pad_sequences(sequence, maxlen=model.input_shape[1], padding='pre')

    for line_num in range(num_lines):
        # For each line after the first one, don't add the starting word again
        if line_num > 0:
            line = ""  # Start the line without the starting word
        else:
            line = start_word  # First line should start with the starting word

        for _ in range(num_words_per_line - 1):  # Subtract 1 because we already have the first word in the line
            # Predict the next word probabilities
            pred_probs = model.predict(sequence)[0]

            # Apply temperature scaling
            pred_probs = np.asarray(pred_probs).astype('float64')
            pred_probs = np.log(pred_probs) / temperature
            pred_probs = np.exp(pred_probs) / np.sum(np.exp(pred_probs))  # Normalize to get probabilities

            # Apply the selected sampling method
            if sampling_method == 'top_k':
                predicted_word_index = top_k_sampling(pred_probs, k=k)
            elif sampling_method == 'top_p':
                predicted_word_index = top_p_sampling(pred_probs, p=p)
            else:
                # Default to random choice based on probabilities
                predicted_word_index = np.random.choice(range(len(pred_probs)), p=pred_probs)

            predicted_word = tokenizer.index_word.get(predicted_word_index, "")

            if predicted_word == "":  # If the predicted word is empty, break the loop
                break

            line += ' ' + predicted_word

            # Update the sequence with the newly predicted word
            sequence = np.append(sequence[:, 1:], predicted_word_index).reshape(1, -1)

        # Add the generated line to the poem
        poem += '\n' + line

    return poem


In [23]:
# Prompt the user to enter the starting word, number of words per line, number of lines, and sampling method
start_word = input("Enter the starting word: ")
num_words_per_line = int(input("Enter the number of words per line: "))
num_lines = int(input("Enter the number of lines: "))
temperature = float(input("Enter the temperature (e.g., 0.5 for more creative results): "))
sampling_method = input("Enter the sampling method ('top_k' or 'top_p'): ")

# Set parameters for top-k or top-p sampling
k = 5  # For top-k sampling, the number of words to consider
p = 0.9  # For top-p sampling, the cumulative probability threshold

# Generate the poetry based on the input
generated_poem = generate_poetry(model, tokenizer, start_word, num_words_per_line=num_words_per_line, num_lines=num_lines, temperature=temperature, sampling_method=sampling_method, k=k, p=p)

print("\nGenerated Poetry:\n")
print(generated_poem)


Enter the starting word: use
Enter the number of words per line: 8
Enter the number of lines: 5
Enter the temperature (e.g., 0.5 for more creative results): 1.4
Enter the sampling method ('top_k' or 'top_p'): 0.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s